In [1]:
import darts
import pandas as pd

series = darts.TimeSeries.from_dataframe(pd.read_parquet('data.parquet'))

train_series, val_series = series.split_after(0.8)

In [2]:
from darts.dataprocessing.transformers import Scaler

scaler = Scaler()
train_series = scaler.fit_transform(train_series)
val_series = scaler.transform(val_series)

In [4]:
from darts.models import TransformerModel

model = TransformerModel(
    input_chunk_length=24 * 2 * 4, # 2 days at 15 min intervals
    output_chunk_length=24 * 4, # 1 day at 15 min intervals
)

target_col = 'Level Durham New Elvet Bridge'
train_target = train_series[target_col]
train_past_covariates = train_series.drop_columns(target_col)

val_target = val_series[target_col]
val_past_covariates = val_series.drop_columns(target_col)

model.fit(
    series=train_target, 
    past_covariates=train_past_covariates, 
    val_series=val_target,
    val_past_covariates=val_past_covariates,
    verbose=True, 
)


/home/ejex/micromamba/envs/river/lib/python3.10/site-packages/torch/nn/modules/transformer.py:286: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name                | Type                | Params
------------------------------------------------------------
0 | criterion           | MSELoss             | 0     
1 | train_metrics       | MetricCollection    | 0     
2 | val_metrics         | MetricCollection    | 0     
3 | encoder             | Linear              | 960   
4 | positional_encoding | _PositionalEncoding | 0     
5 | transformer         | Transformer         | 548 K 
6 | decoder             | Linear              | 6.2 K 
------------------------------------------------------------
555 K     Trainable params
0         Non-trainable params
555 K     Total params
2.223     Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]